In [1]:
import pandas as pd
import json
import requests
import random
from lxml import html
import re
import time

from  util.spider import Spider
from util.http_utility import get_http_headers
from util.user_agent import get_random_ua

# Food4Thought: Russia

url: https://www.russianfood.com/recipes/bytype/?fid=103#rcp_list

The website contains a lot of recipes from different countries, fid=103 is for Russia. 
Russianfood.com has IP blocking, need time sleep... and disable multithreading.

In [5]:
class RussianSpider(Spider): #inherit Spider Class

    def __init__(self, main_page, seeds, listing, attrs, header=None):
        super().__init__(main_page,seeds,listing,attrs,header)

    def scrape_one_item(self,url):
        """To scrape one item from a given url

        Args:
            url (str): url of the item

        Returns:
            dict: dict of the item
        """
        time.sleep(5)
        session = requests.Session() 
        session.headers.update(self.header)

        #add home page url to the obtained item url if item url is not complete
        if self.main_page not in url:
            res = session.get(self.main_page + url)
        else:
            res = session.get(url)

        if res.status_code == 200:
            try:
                #print(res.status_code)
                doc = html.document_fromstring(res.text)
                #set default values for variables
                name, total_time, ingredients, instructions, servings, category, prep_time, cook_time = '','','','','','','',''

                #Name
                if self.check_normalize_space(self.attrs['name']):
                    name = doc.xpath(self.attrs['name'])
                else: name = doc.xpath(self.attrs['name'])[0]

                #Total Time
                if self.check_normalize_space(self.attrs['total_time']):
                    total_time = doc.xpath(self.attrs['total_time'])
                else:  total_time = doc.xpath(self.attrs['total_time'])[0]

                #ingredients
                ingredients = doc.xpath(self.attrs['ingredients'])
                clean_ingredients = []
                for ing in ingredients:
                    ing = ing.replace('\t','')
                    ing = ing.replace('\r','')
                    ing = ing.replace('\n','')
                    clean_ingredients.append(ing)
                #print(clean_ingredients)
                ingredients = ''.join(clean_ingredients)
                ingredients = ingredients.strip()
                # ingredient_str = ''
                # for ing in ingredients:
                #     if  ' ' not in str(ing):
                #         ingredient_str += ing
                # ingredients = ingredient_str
                

                #instructions
                instructions = doc.xpath(self.attrs['instructions'])
                #print(instructions)
                instructions= ''.join(instructions)
                
                # instructions_str = ''
                # for ins in instructions:
                #     if  ' ' not in str(ins): 
                #         instructions_str += ins
                # instructions = instructions_str

                # if self.check_normalize_space(self.attrs['instructions']):
                #     instructions = doc.xpath(self.attrs['instructions'])
                # else:  instructions = doc.xpath(self.attrs['instructions'])[0]

                #servings
                if self.attrs['servings']:
                    if self.check_normalize_space(self.attrs['servings']):
                        servings = doc.xpath(self.attrs['servings'])
                    else: servings = doc.xpath(self.attrs['servings'])[0]

                #category
                if self.attrs['category']:
                    if self.check_normalize_space(self.attrs['category']):
                        category = doc.xpath(self.attrs['category'])
                    else: category = doc.xpath(self.attrs['category'])[0]

                #prep time
                if self.attrs['prep_time']:
                    if self.check_normalize_space(self.attrs['prep_time']):
                        prep_time = doc.xpath(self.attrs['prep_time'])
                    else: prep_time = doc.xpath(self.attrs['prep_time'])[0]
                
                #cooking time
                if self.attrs['cook_time']:
                    if self.check_normalize_space(self.attrs['cook_time']):
                        cook_time = doc.xpath(self.attrs['cook_time'])
                    else: cook_time = doc.xpath(self.attrs['cook_time'])[0]
                
                return {'name': name, 'total_time': total_time, 'ingredients': ingredients, 'instructions': instructions, 'servings': servings,
                'category': category, 'prep_time': prep_time, 'cook_time': cook_time,}
            except Exception as e:
                raise Exception('something is wrong for item: {}'.format(url)) from e


        else:
            raise Exception('Cannot open one menu url, status code = {}'.format(res.status_code))


In [3]:
attrs = {
        'name':         'normalize-space(//title)',
        'ingredients':  '//tr[contains(@class,"ingr_tr_")]//text()',
        'total_time':   'normalize-space(//div[@class="sub_info"]/div[@class="el"][2])',
        'instructions': '//div[@class="step_n"]//p//text()',
        'servings':     'normalize-space(//div[@class="sub_info"]/div[@class="el"][1])',
        'category':     '',
        'prep_time':    '',
        'cook_time':    '',
}

listing={'items': '//div[contains(@class,"recipe_list_new")]//div[contains(@class,"title")]/a/@href', 'next': { 'next_page_str': '&page={}', 'type': 'url'}}
seeds = ['https://www.russianfood.com/recipes/bytype/?fid=103']

In [15]:
#setup variables
user_agent = get_random_ua()
custom_header = { #setup custom header because romania requires certain headers
        'referer': 'https://www.google.com/',
        'Accept-Language': '*',
        'Accept-Encoding': '*',
        'Accept': '*',
        'user-agent': user_agent}
ru_spider= RussianSpider('https://www.russianfood.com/', seeds= seeds, listing =listing,attrs= attrs, header=custom_header)
#print(romania_spider.attrs)
ru_spider.scrape_one_item('https://www.russianfood.com/recipes/recipe.php?rid=164402') #try out scraping one item

Exception: Cannot open one menu url, status code = 403

In [9]:
result_list = ru_spider.start_scrape(max_pages=65,multithread=False) #max page is only 65, have to disable multithreading because of  blocking

spider is scraping page: 1
spider is scraping page: 2
spider is scraping page: 3
spider is scraping page: 4
spider is scraping page: 5
spider is scraping page: 6
spider is scraping page: 7
something is wrong for item: /recipes/recipe.php?rid=62918


In [11]:
result_list

[{'name': 'Рецепт: Салат "Русский барин" на RussianFood.com',
  'total_time': '\xa01 час 30 мин (ваши 30 мин)',
  'ingredients': 'Печень куриная отварная - 160 гКартофель отварной - 140 гГрибы белые сушеные (взвешивал отварными) - 100 гМорковь - 80 гЛук - 100 гОгурец соленый - 120 гПерепелиные яйца - 6 шт.Зелень петрушки - по вкусуМайонез - 100 г + 60 г для украшенияОстрый соус Табаско - 40 капельСоус бальзамик для украшенияЛук красный для украшенияМасло для обжаривания',
  'instructions': 'Печень кубиком.Морковь соломкой.Готовый салат из куриной печени.',
  'servings': '\xa02 порции по 500 ККал',
  'category': '',
  'prep_time': '',
  'cook_time': ''},
 {'name': 'Рецепт: Паштет из копчёной скумбрии со сливками и яйцом на RussianFood.com',
  'total_time': '\xa025 мин (ваши 25 мин)',
  'ingredients': 'Скумбрия копчёная - 300 гСливки (жирностью не менее 30\u2009%) - 150 гЯйца - 2 шт.Перец чёрный молотый  - по вкусу',
  'instructions': 'Подготовить продукты.\r\nСливки лучше использовать ж

In [12]:
result_df = pd.DataFrame(result_list)

In [13]:
result_df

,name,total_time,ingredients,instructions,servings,category,prep_time,cook_time
0,"Рецепт: Салат ""Русский барин"" на RussianFood.com",1 час 30 мин (ваши 30 мин),Печень куриная отварная - 160 гКартофель отвар...,Печень кубиком.Морковь соломкой.Готовый салат ...,2 порции по 500 ККал,,,
1,Рецепт: Паштет из копчёной скумбрии со сливкам...,25 мин (ваши 25 мин),Скумбрия копчёная - 300 гСливки (жирностью не ...,Подготовить продукты.\r\nСливки лучше использо...,5 порций,,,
2,Рецепт: Картофельники по-московски на RussianF...,1 час (ваши 30 мин),Картофель - 8-10 шт.Яйцо куриное - 2 шт.Мука -...,,4 порции,,,
3,"Рецепт: Капуста, квашенная с яблоками на Russi...",3 дня (ваши 40 мин),Капуста - 1 кгЯблоки - 250-300 гМорковь - 80 г...,Подготовить продукты по списку.Капусту освобод...,4 порции,,,
4,Рецепт: Блинчики заварные молочно-йогуртовые н...,1 час,Йогурт - 250 млМолоко - 250 млМука - 8-9 ст. л...,Продукты для блинчиков перед вами.Яйца вбить в...,6 порций,,,
...,...,...,...,...,...,...,...,...
295,Рецепт: Старинная русская шарлотка из пшенично...,2 часа (ваши 35 мин),Хлеб пшенично-ржаной (дарницкий) – 1 буханка (...,Подготавливаем продукты. \r\nВ процессе пригот...,6 порций,,,
296,Рецепт: Блины с маком на RussianFood.com,40 мин,Яйца куриные - 2 шт.Молоко - 300 гМука пшеничн...,Продукты для блинов с маком перед вами.Как при...,4 порции,,,
297,"Рецепт: Голубцы из пекинской капусты, с картоф...",,Картофель - 4 шт.Капуста пекинская - 1 шт.Беко...,Подготовим продукты для голубцов из пекинский ...,40 мин,,,
298,Рецепт: Слоёные пышки (лепёшки) на кефире на R...,20 мин (ваши 20 мин),Кефир - 350 млМука - 400 г + на подсыпкуСода -...,Как приготовить слоёные пышки (лепёшки) на кеф...,3 порции,,,


In [14]:
result_df.to_csv('data/russia/russian_food_com.csv')

I got blocked from accessing this website whilst I was scraping the 7th page. Changing IP addresses with VPN gives the same problem.